<div style="width: 100%; clear: both;">
<div style="float: left; width: 50%;">
<img src="http://www.uoc.edu/portal/_resources/common/imatges/marca_UOC/UOC_Masterbrand.jpg", align="left">
</div>
</div>
<div style="float: right; width: 50%;">
<p style="margin: 0; padding-top: 22px; text-align:right;">M2.856 · Anàlisi de sentiments i xarxes socials</p>
<p style="margin: 0; text-align:right;">Màster universitari en Ciència de dades (Data science)</p>
<p style="margin: 0; text-align:right; padding-button: 100px;">Estudis d'Informàtica, Multimèdia i Telecomunicació</p>
</div>
</div>
<div style="width: 100%; clear: both;">
<div style="width:100%;">&nbsp;</div>

# Anàlisi de sentiments i xarxes socials
## PLA4: Representació de xarxes socials com a grafs

## Ús de grafs per a representar dades de xarxes socials

En el capítol 2 del llibre *Anàlisis de datos de redes sociales*, *Introducción a la teoría de grafos*, hem descrit què són els grafs, els diferents tipus de grafs que hi ha i les propietats bàsiques que tenen. En aquest notebook, veurem com podem representar en forma de graf les dades de xarxes socials que hem adquirit amb anterioritat (usant qualsevol dels tres mètodes descrits en el notebook anterior)mitjançant la llibreria de Python  [networkx](https://networkx.github.io/).

## 1. Representació bàsica 

[networkx](https://networkx.github.io/) és una la llibreria de Python per a crear i estudiar xarxes. Disposa de classes pròpies per a representar els grafs, que ja inclouen mètodes que implanten alguns dels algorismes més comuns per a analitzar, visualitzar i exportar grafs.

Networkx disposa de funcions d'importació que permeten crear un graf a partir de les dades d’un fitxer de text amb uns formats concrets. Així, per exemple, les dades de Google+ que hem descarregat anteriorment del [Network repository](http://networkrepository.com/soc.php), es trobaven en format «llista d'arestes», on cada línia del fitxer descriu una aresta del graf. Podem utilitzar la funció [`read_edgelist`](https://networkx.github.io/documentation/networkx-2.0/reference/readwrite/generated/networkx.readwrite.edgelist.read_edgelist.html#networkx.readwrite.edgelist.read_edgelist) per a crear un graf de networkx a partir del fitxer descarregat:

In [1]:
# Importem la llibreria networkx.
import networkx as nx

# Obrim el fitxer descarregat.
with open("./data/soc-gplus.edges", 'rb') as gp_file:
    
    # Creem el graf.
    g = nx.read_edgelist(gp_file, comments="%")
    
    # Mostrem el nombre de nodes i arestes del graf.
    print("El graf té {} nodes i {} aristes".format(
        g.number_of_nodes(), g.number_of_edges()))

El graf té 23628 nodes i 39194 aristes


Fixeu-vos que cal especificar el caràcter que s'utilitza per a indicar quines línies del fitxer corresponen a comentaris per aconseguir que el graf es carregui correctament, ja que, per defecte, la funció espera coixinets  `#` i el fitxer que hem importat fa servir  `%`.

A més de llegir una llista d'arestes amb  `read_edgelist`, networkx disposa d'altres funcions de càrrega de dades de fitxers que permeten importar fitxers en altres formats. Alguns dels més populars són [GML](https://networkx.github.io/documentation/networkx-2.0/reference/readwrite/generated/networkx.readwrite.gml.read_gml.html#networkx.readwrite.gml.read_gml), [graphml](https://networkx.github.io/documentation/networkx-2.0/reference/readwrite/generated/networkx.readwrite.graphml.read_graphml.html#networkx.readwrite.graphml.read_graphml), o [gexf](https://networkx.github.io/documentation/networkx-2.0/reference/readwrite/generated/networkx.readwrite.gexf.read_gexf.html#networkx.readwrite.gexf.read_gexf). En la documentació de networkx trobareu la [llista completa de formats](https://networkx.github.io/documentation/networkx-2.0/reference/readwrite/index.html) pels quals hi ha funcions d'importació.

Si no disposem d'un fitxer amb totes les dades del graf, podem optar per crear un graf buit i anar-hi afegint els nodes i les arestes a mesura que anem obtenint les dades. Aquesta seria una opció per a crear un graf a partir de dades que ens vagin retornant les consultes successives a una API.

Les classes  [`Graph`](https://networkx.github.io/documentation/networkx-2.0/reference/classes/graph.html#networkx.Graph) y [`DiGraph`](https://networkx.github.io/documentation/networkx-2.0/reference/classes/digraph.html#networkx.DiGraph) representen grafs simples no-dirigits i dirigits, respectivament. Els mètodes [`add_node`](https://networkx.github.io/documentation/networkx-2.0/reference/classes/generated/networkx.Graph.add_node.html#networkx.Graph.add_node) i [`add_edge`](https://networkx.github.io/documentation/networkx-2.0/reference/classes/generated/networkx.Graph.add_edge.html#networkx.Graph.add_edge) permeten afegir un node o una aresta a tots dos tipus de grafs.  

Vegem un exemple de com podem crear un graf amb dades de relacions de seguiment d'usuaris de Twitter usant les funcions de l'API de Twitter que hem vist al notebook anterior i la classe  `DiGraph` de networkx que acabem de presentar.

En primer lloc, recuperem el codi d'autenticació de l'API de Twitter:

In [2]:
# Importem la llibreria tweepy.
import tweepy

# Definim la variable API, que emmagatzemarà l'objecte tweepy.API inicialitzat amb les nostres credencials.
api = None

# Definim la funció get_twitter_api, que retornarà l'objecte tweepy.API. La funció crearà
# un objecte tweepy.API si aquest no s'ha creat prèviament (o si ho indiquem explícitament 
# amb reset=True), o bé retornarà l'objecte creat anteriorment.
def get_twitter_api(reset=False, consumer_key=None, consumer_secret=None, access_token=None, access_secret=None):
    global api
    if not api or reset:
        print("Initializing tweepy API...")
        auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
        auth.set_access_token(access_token, access_secret)
        api = tweepy.API(auth)
    return api

# Creem un objecte tweepy.API amb la funció get_twitter_api.
# IMPORTANT: cal incloure les credencials d'accés que hàgiu obtingut en crear la vostra App
# per a executar la resta d'exemples  del notebook.
api = get_twitter_api(consumer_key = 'tuzlLXJ7Cjcw9ysoeEhT60ENh', 
                      consumer_secret = 'cO9hqRaP94IP66ssVqHHqVyibrLMQ1et8XHbhczy2eBW6rFFiI',
                      access_token = '1029309220234186752-GikHFK2syTi2kiH0bVx8fMNhdP8IDJ',
                      access_secret = 'AJIuznR3PXwPh30GZ8YYqyQfpTyjkZrnu5bglzGUSaN5F'
                      )


Initializing tweepy API...


Després, implementem dues funcions que incorporen la lògica del nostre script. La primera, `add_connections_from`, recupera els *followers* i els *friends* d'un usuari de Twitter i afegeix els nodes i les arestes al graf. La segona, `bfs_from_seed`, implementa una cerca *Breadth First Search* a partir d'un node inicial, que rep com a paràmetre (consulteu el capítol 2 del llibre *Análisis de datos de redes sociales* si no recordeu com funciona aquest algorisme).

In [3]:
def add_connections_from(g, new_nodes, userid):
    """
    Adds followers and friends from user with id userid to the graph g.
    
    :param g: networkx graph
    :param new_nodes: list of nodes where newly discovered nodes will be added
    :param userid: integer, Twitter user id of the node that will be visited
    """
    
    try:
        # get userid's user followers
        followers = api.followers_ids(userid)
        print("{} followers retrieved".format(len(followers)))
        for follower in followers:
            # if the follower is unknown, add follower to the graph and to new_nodes list
            if follower not in g.nodes():
                g.add_node(follower)
                new_nodes.append(follower)
            # add edge to the graph
            g.add_edge(userid, follower)

        # get userid's user friends
        friends = api.friends_ids(userid)
        print("{} friends retrieved".format(len(friends)))
        for friend in friends:
            # if the friend is unknown, add friend to the graph and to new_nodes list
            if friend not in g.nodes():
                g.add_node(friend)
                new_nodes.append(friend)
            # add edge to the graph
            g.add_edge(friend, userid)
    except tweepy.TweepError:
        # If we can not obtain data from this user, skip it
        print("Failed to run the command on that user, skipping...")
    except tweepy.RateLimitError:
        # If we hit the rate limit, wait for 15 minutes before calling the API again
        print("Rate limit reached. Waiting for 15 minutes...")
        time.sleep(15 * 60)

    return g, new_nodes

def bfs_from_seed(seed_id, number_of_crawled_users=3):
    """
    Performs a bfs search starting from user seed_id and stopping after having
    processed number_of_crawled_users users.
    
    :param seed_id: int, user id of the starting user
    :param number_of_crawled_users: int, number of users to crawl
    """

    print("Starting to crawl at node {}\n".format(seed_id))
    
    # create an empty graph and add seed node
    g = nx.DiGraph()
    g.add_node(seed_id)
    
    # create a list to store newly discovered users
    new_nodes = []
    
    # crawl number_of_crawled_users users
    for _ in range(number_of_crawled_users):
        print("New user processed {}".format(seed_id))
        # add friends of followers of crawled user to the graph
        g, new_nodes = add_connections_from(g, new_nodes, seed_id)
        print("Waiting list has now {} users".format(len(new_nodes)))
        print("Graph has now {} nodes and {} edges\n".format(
            g.number_of_nodes(), g.number_of_edges()))
        # get next user to crawl
        seed_id = new_nodes.pop(0)
        
    return g, new_nodes




In [4]:
bfs_from_seed(2775195514)

Starting to crawl at node 2775195514

New user processed 2775195514
1760 followers retrieved
144 friends retrieved
Waiting list has now 1824 users
Graph has now 1825 nodes and 1904 edges

New user processed 1037262684029370368
55 followers retrieved
278 friends retrieved
Waiting list has now 2098 users
Graph has now 2100 nodes and 2236 edges

New user processed 232668523
Failed to run the command on that user, skipping...
Waiting list has now 2097 users
Graph has now 2100 nodes and 2236 edges



(<networkx.classes.digraph.DiGraph at 0x108885780>,
 [1119737786557456385,
  799963701604454400,
  795676354977079296,
  15039203,
  140694740,
  1144221712336244736,
  2325217039,
  219635897,
  97070739,
  2410354324,
  2396037342,
  2312793855,
  1140319788306259969,
  1067809214624669697,
  426698976,
  813611378,
  203652936,
  1339878091,
  1000249440765169665,
  103377307,
  132204002,
  4155536837,
  1164611,
  103131326,
  862918790958518273,
  4897204209,
  364449057,
  761783419,
  207994318,
  292609406,
  1005786690353598465,
  289291773,
  4053496954,
  302207857,
  223082075,
  636126399,
  632844131,
  245022854,
  1109816501455020033,
  2778890025,
  1001155508,
  4811665277,
  109899149,
  1131566718118244352,
  706450767021002752,
  409395187,
  196665734,
  4369594635,
  860127057522688000,
  969251488781434880,
  2242599391,
  1130832192706416640,
  299526368,
  129596166,
  315376679,
  104653002,
  2331890599,
  260355276,
  1127935596863873024,
  106556090677879

## 2. Grafs amb atributs

Tant els grafs com les arestes i els nodes poden contenir atributs a networkx. Aquests atributs es poden utilitzar, per exemple, per a emmagatzemar metadades del graf, pesos o *timestamps* a les arestes, propietats dels nodes, etc. 

Els atributs es poden assignar en el moment de la creació del graf, el node o l'aresta, o bé posteriorment:

In [6]:
# Creem el graf g amb els atributs 'network' i 'seed'.
g = nx.Graph(network='Twitter', seed=2775195514)
print("Els atributs del graf són: {}".format(g.graph))
print("L'atribut network té com a valor: {}".format(g.graph['network']))
print("L'atribut seed té com a valor: {}".format(g.graph['seed']))

Els atributs del graf són: {'network': 'Twitter', 'seed': 2775195514}
L'atribut network té com a valor: Twitter
L'atribut seed té com a valor: 2775195514


In [7]:
# Creem un graf g buit i afegim després els atributs  'network' i 'seed'.
g = nx.Graph()
print('Els atributs del graf són: {}'.format(g.graph))
g.graph['network'] = "Twitter"
g.graph['seed'] = 2775195514
print('Els atributs del graf són: {}'.format(g.graph))

Els atributs del graf són: {}
Els atributs del graf són: {'network': 'Twitter', 'seed': 2775195514}


In [8]:
# Afegim un node amb atributs al graf.
g.add_node(1, day='monday')
print('Els atributs dels nodes són: {}'.format(g.nodes(data=True)))

Els atributs dels nodes són: [(1, {'day': 'monday'})]


In [9]:
# Afegim un node sense atributs i els assignem després.
g.add_node(2)
print('Els atributs dels nodes són: {}'.format(g.nodes(data=True)))
g.nodes[2]['day'] = 'tuesday'
print('Els atributs dels nodes són: {}'.format(g.nodes(data=True)))

Els atributs dels nodes són: [(1, {'day': 'monday'}), (2, {})]
Els atributs dels nodes són: [(1, {'day': 'monday'}), (2, {'day': 'tuesday'})]


In [10]:
# Afegim una aresta amb atributs al graf.
g.add_edge(1, 2, friendship_created_at='2018-07-15')
print('Els atributs de les arestes són: {}'.format(g.edges(data=True)))

Els atributs de les arestes són: [(1, 2, {'friendship_created_at': '2018-07-15'})]


In [11]:
# Afegim una aresta sense atributs i els afegim després.
g.add_node(3)
g.add_edge(1, 3)
print('Els atributs de les arestes són: {}'.format(g.edges(data=True)))
g[1][3]['friendship_created_at'] = '2018-04-23'
print('Els atributs de les arestes són: {}'.format(g.edges(data=True)))

Els atributs de les arestes són: [(1, 2, {'friendship_created_at': '2018-07-15'}), (1, 3, {})]
Els atributs de les arestes són: [(1, 2, {'friendship_created_at': '2018-07-15'}), (1, 3, {'friendship_created_at': '2018-04-23'})]


Fixeu-vos que per poder afegir una segona aresta al graf, hem hagut d'afegir un tercer node. Això és així perquè la classe  [`Graph`](https://networkx.github.io/documentation/networkx-2.0/reference/classes/graph.html#networkx.Graph) representa grafs simples, en els quals no s'admeten arestes repetides. Per a representar les relacions d'amistat d'una xarxa social, normalment serà suficient usar aquest tipus de grafs. De totes maneres, de vegades pot ser necessari permetre arestes repetides. És el cas, per exemple, d'un graf que representi *replies* entre usuaris de Twitter: un usuari pot respondre més d'una vegada a un altre usuari. La classe [`MultiGraph`](https://networkx.github.io/documentation/networkx-2.0/reference/classes/multigraph.html#multigraph-undirected-graphs-with-self-loops-and-parallel-edges) de networkx ens permet representar aquests grafs.

In [12]:
# creem un multigraf.
g = nx.MultiGraph()

# Afegim dos nodes i dues arestes entre ells.
g.add_nodes_from([1,2])
g.add_edge(1,2, reply_sent_at='2018-07-15')
g.add_edge(1,2, reply_sent_at='2018-07-18')
print('Les arestes són: {}'.format(g.edges(data=True)))

Les arestes són: [(1, 2, {'reply_sent_at': '2018-07-15'}), (1, 2, {'reply_sent_at': '2018-07-18'})]


Encara que networkx no té suport natiu per a grafs dinàmics, la classe `MultiGraph` pot ser útil per a representar aquests grafs. Usant atributs, podem especificar els moments en els quals els nodes i les arestes són creats i/o eliminats del graf.